In [1]:
import sys
import os

# Get the absolute path of the project root (adjust as needed)
project_root = os.path.abspath("../..")  # Moves one level up to project root

# Add to sys.path if not already present
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
# Load all import parameters
from models.input.pt_imported_simple_parameter import PtImportedSimpleParameter
from models.input.pt_imported_range import PtImportedRange

from models.output.pt_sdp3_range import PtSDP3Range
from models.output.pt_sdp3_parameter import PtSDP3Parameter

In [3]:
# load all import parameters


from pathlib import Path

all_files = Path("./data/Function-Parameters/APS2/Imported/Parameters").glob("*.xml")

# make sure their names start with PtImportedSimpleParameter
imported_parameter_files = [
    file for file in all_files if file.name.startswith("PtImportedSimpleParameter")
]

funciton_properties_details = ""

imported_parameters: list[PtImportedSimpleParameter] = []

for file in imported_parameter_files:
    with file.open("rb") as f:
        xml_data = f.read()
        obj = PtImportedSimpleParameter.from_xml(xml_data)
        model_json = obj.model_dump_json(indent=2)
        imported_parameters.append(obj)

# print(physical_quantities)

In [4]:
# load all range parameters
all_files = Path("./data/Function-Parameters/APS2/Imported/Ranges").glob("*.xml")

# make sure their names start with PtImportedRange
imported_range_files = [
    file for file in all_files if file.name.startswith("PtImportedRange")
]

imported_ranges: list[PtImportedRange] = []

for file in imported_range_files:
    with file.open("rb") as f:
        xml_data = f.read()
        obj = PtImportedRange.from_xml(xml_data)
        model_json = obj.model_dump_json(indent=2)
        imported_ranges.append(obj)

In [5]:
imported_parameters

[PtImportedSimpleParameter(name='ExtraDryingPerformance', ecuSystemFamily=RefElement(ref='PtEcuSystemFamily', name='APS'), ecuSystemExecution=RefElement(ref='PtEcuSystemExecution', name='APS2'), serverExecution=RefElement(ref='PtServerExecution', name='APS2'), importedSimpleParameter=ImportedSimpleParameter(name='ExtraDryingPerformance', description='Parameter to select different drying performance for high capacity device\r\n1 -> old drying performance setting\r\n2 -> new drying performance setting\r\n3 to 4 -> reserved for Future, new drying performance setting is used', Unit='-', rangeRef=RefElement(ref='PtImportedRange', name='ExtraDryingPerformanceRange'), UserFunction='193'))]

In [6]:
from models.input.physical_quantity import PhysicalQuantity

# load all the physical quantities from the ./data/Function-Parameters/PhysicalQuantity/ folder
from pathlib import Path

physical_quantity_files = Path("./data/Function-Parameters/PhysicalQuantity").glob(
    "*.xml"
)

# make sure their names start with PhysicalQuantity
physical_quantity_files = [
    file for file in physical_quantity_files if file.name.startswith("PhysicalQuantity")
]

physical_quantities = []
token_count = 0
print("total files: ", len(physical_quantity_files))

physical_quantities_details = ""

for file in physical_quantity_files:
    with file.open("rb") as f:
        xml_data = f.read()
        obj = PhysicalQuantity.from_xml(xml_data)

        json_data = obj.model_dump_json(indent=2)
        # add the name of the physical quantity to the details
        physical_quantities_details += "\n\nPHYSICAL QUANTITY " + obj.name + "\n"

        ## loop through the units and add them to the details
        for unit in obj.unit:
            physical_quantities_details += unit.name + "\n"

        physical_quantities.append(obj)


print(physical_quantities_details)



print(f"Token count: {token_count}")
# print(physical_quantities)

total files:  83


PHYSICAL QUANTITY Rms
rpm/mg/str


PHYSICAL QUANTITY PartsPerMillion
ppm
-


PHYSICAL QUANTITY SNR
dB


PHYSICAL QUANTITY Volume_litres
l


PHYSICAL QUANTITY SignalStrength
%
-


PHYSICAL QUANTITY Time_year
-


PHYSICAL QUANTITY AngularVelocity
degree/s
rad/s


PHYSICAL QUANTITY Angle
degree
deg
rad


PHYSICAL QUANTITY Torque
Nm


PHYSICAL QUANTITY Flow_liters_per_seconds
l/s
ls


PHYSICAL QUANTITY Current
A
mA


PHYSICAL QUANTITY millimeter_per_degree
mm/deg


PHYSICAL QUANTITY Time_min
min
s


PHYSICAL QUANTITY Acceleration
m/s2
km/h/s


PHYSICAL QUANTITY Flow
l/h
l/min


PHYSICAL QUANTITY Impulses
I/km
I/m
pulses/m
pulses/km


PHYSICAL QUANTITY NumberOfRevolutionsPerSecond
rpm/s


PHYSICAL QUANTITY Energy_Wh
Wh


PHYSICAL QUANTITY Length_m
m


PHYSICAL QUANTITY Pressure
bar
Bar
mbar
mBar
Pa
kPa
kpa
hPa


PHYSICAL QUANTITY Length_cm
cm


PHYSICAL QUANTITY Length_mm
mm
m


PHYSICAL QUANTITY Volume_cm3
cm3


PHYSICAL QUANTITY Concentration_g_per_kg
g/kg


PHYSICAL QU

In [7]:
from openai import OpenAI
from pydantic import BaseModel

# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"


client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

term = "CameraSystems"


class FunctionParameterDetails(BaseModel):
    description: str
    physical_quantity: str
    reason: str


def generate_output_parameter(
    function_parameter: PtImportedSimpleParameter,
    function_range: PtImportedRange,
    search_results: str,
):
    chat_response = client.chat.completions.create(
        model="NovaSky-AI/Sky-T1-32B-Flash",
        messages=[
            {
                "role": "system",
                "content": f"""
            You are a technical writer for an automotive company. You are familiar with various terms used in automotive documentation.
            Given the details about a function paramter, you need to return description and physical quantitiy of the parameter.

            ========================================
            physical quantities:
            {physical_quantities_details}
            ========================================
            ========================================
            A vector search result:
            {search_results}
            ========================================

            RULES:

            1. Decide the path where we should add this group in the Function Adjust Tree.
            2. Always choose the most specific path.
            3. Return a small reason for your choice.
            """,
            },
            {
                "role": "user",
                "content": f"""
                function_parameter: { function_parameter.model_dump_json(indent=2)}
                function_range: { function_range.model_dump_json(indent=2)}
                    """,
            },
        ],
        temperature=0.5,
        extra_body={"guided_json": FunctionParameterDetails.model_json_schema(), "top_p": 0.9},
    )


    return FunctionParameterDetails.model_validate_json(chat_response.choices[0].message.content)


In [9]:
from database.database import qclient, oclient
from models.common import RefElement
from models.output.pt_sdp3_parameter import PresentationText
from utils import get_tokens


def get_dense_vector(document):
    """
    Convert sparse vector to dense format.
    """
    response = oclient.embeddings(
        model="nomic-embed-text", prompt=" ".join(get_tokens(document))
    )
    embeddings = response["embedding"]

    return embeddings


def process_imported_parameter(imported_parameter: PtImportedSimpleParameter):
    # load associated range

    # find the associated range
    range_name = imported_parameter.importedSimpleParameter.rangeRef.name

    # find the range object
    range_obj = next(
        (r for r in imported_ranges if r.name == range_name), None
    )  # type: ignore

    print(f"Range name: {range_name}")
    print(f"Range object: {range_obj}")

    # create lookup document
    lookup_document = imported_parameter.name + "\n"
    lookup_document += imported_parameter.importedSimpleParameter.description + "\n"

    dense_embedding = get_dense_vector(lookup_document)

    result = qclient.query_points(
        collection_name="function_parameters",
        query=dense_embedding,
        using="text-dense",
        limit=3,
    )

    results_string = ""

    for points in result.points:
        if points is None or points.payload is None:
            continue
        # get the document from the result
        result_doc = points.payload["json"]

        results_string += f"{result_doc} \n"

    llm_data = generate_output_parameter(imported_parameter, range_obj, results_string)
    print(f"LLM data: {llm_data}")
    # output data parameter

    output_data = PtSDP3Parameter(
        name=imported_parameter.name,
        ecuSystemFamily=imported_parameter.ecuSystemFamily,
        ecuSystemExecution=imported_parameter.ecuSystemExecution,
        serverExecution=imported_parameter.serverExecution,
        categoryCondition=RefElement(
            ref="CategoryCondition",
            name="-",
        ),
        description=PresentationText(edt="nfTxt", value=llm_data.description),
        physicalQuantity=RefElement(
            ref="PhysicalQuantity",
            name=llm_data.physical_quantity,
        ),
        namePresentation=PresentationText(edt="nfTxt", value=imported_parameter.name),
        importedSimpleParameterRef=RefElement(
            ref="ImportedSimpleParameter",
            name=imported_parameter.name,
        ),
    )

    # save the output data in xml format
    output_data_xml = output_data.to_xml(
        pretty_print=True,
        encoding="UTF-8",
        xml_declaration=True,
    )

    # export the range in the xml format

    if range_obj is None:
        print(f"Range object not found for {range_name}")
        return

    range_data = PtSDP3Range(
        name=range_obj.name,
        ecuSystemFamily=imported_parameter.ecuSystemFamily,
        ecuSystemExecution=imported_parameter.ecuSystemExecution,
        serverExecution=imported_parameter.serverExecution,
        importedRangeRef=RefElement(
            ref="ImportedRange",
            name=range_obj.name,
        ),
        discreteValueRef=[],
    )

    for value in range_obj.importedDiscreteValue:
        range_data.discreteValueRef.append(
            RefElement(
                ref="DiscreteValue",
                name=value.name,
            )
        )


    range_data_xml = range_data.to_xml(
        pretty_print=True,
        encoding="UTF-8",
        xml_declaration=True,
    )

    print(f"Output data XML: {output_data_xml}")
    
    print(f"Range data XML: {range_data_xml}")

process_imported_parameter(imported_parameters[0])

Range name: ExtraDryingPerformanceRange
Range object: name='ExtraDryingPerformanceRange' ecuSystemFamily=EcuSystemFamily(ref='PtEcuSystemFamily', name='APS') ecuSystemExecution=EcuSystemExecution(ref='PtEcuSystemExecution', name='APS2') serverExecution=ServerExecution(ref='PtServerExecution', name='APS2') importedDiscreteValue=[ImportedDiscreteValue(name='Off'), ImportedDiscreteValue(name='Max')]


LLM data: description='Parameter to select different drying performance settings for high capacity devices. The options are: 1 for old drying performance, 2 for new drying performance, and values 3 to 4 are reserved for future use where the new drying performance setting is applied.' physical_quantity='Dimensionless' reason="The parameter is a selection option without any unit, hence it falls under 'Dimensionless'."
Output data XML: b'<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<PtSDP3Parameter>\n  <Name>ExtraDryingPerformance</Name>\n  <EcuSystemFamily ref="PtEcuSystemFamily">APS</EcuSystemFamily>\n  <EcuSystemExecution ref="PtEcuSystemExecution">APS2</EcuSystemExecution>\n  <ServerExecution ref="PtServerExecution">APS2</ServerExecution>\n  <CategoryCondition ref="CategoryCondition">-</CategoryCondition>\n  <NamePresentation edt="nfTxt">ExtraDryingPerformance</NamePresentation>\n  <Description edt="nfTxt">Parameter to select different drying performance settings for high capacity devi